IMPORTO LIBRERÍAS

In [1]:
from pymongo import MongoClient
import pandas as pd
import pandas as pd
from shapely.geometry import Point
import requests
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv("../Data/df_UberEats.csv", index_col = 0)
df.head(3)

,Restaurante,Valoracion,Nota,Delivery_fee,ETA,Tag,Precio,Direccion,Barrio,Lat,Lon
0,Badsha India Restaurant,200,4.4,1.00,15–35,India,€,"Calle De Embajadores, Madrid",Centro,40.40069,-3.69925
1,TUTTO PASTA!,26,2.8,2.50,40–60,Family Meals,€,"Calle De Núñez De Arce, 11, Madrid",Centro,40.41538,-3.70124
2,Pizza Central,31,3.8,2.99,35–55,Pizza,€,"Plaza Del Emperador Carlos V, 10, Madrid",Centro,40.40861,-3.69288


In [3]:
df['Coordenates'] = df[['Lon', 'Lat']].apply(list, axis=1).tolist()
df.head(3)

,Restaurante,Valoracion,Nota,Delivery_fee,ETA,Tag,Precio,Direccion,Barrio,Lat,Lon,Coordenates
0,Badsha India Restaurant,200,4.4,1.00,15–35,India,€,"Calle De Embajadores, Madrid",Centro,40.40069,-3.69925,"[-3.69925, 40.40069]"
1,TUTTO PASTA!,26,2.8,2.50,40–60,Family Meals,€,"Calle De Núñez De Arce, 11, Madrid",Centro,40.41538,-3.70124,"[-3.70124, 40.41538]"
2,Pizza Central,31,3.8,2.99,35–55,Pizza,€,"Plaza Del Emperador Carlos V, 10, Madrid",Centro,40.40861,-3.69288,"[-3.69288, 40.40861]"


In [4]:
coordenadas = [-73.93, 40.82]

In [5]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [6]:
punto = type_point(coordenadas)
punto

{'type': 'Point', 'coordinates': [-73.93, 40.82]}

In [7]:
tp = []
for i in df["Coordenates"]:
    tp.append(type_point(i))
df['type_point'] = tp

In [8]:
df['type_point'][0]

{'type': 'Point', 'coordinates': [-3.69925, 40.40069]}

In [9]:
df.head(5)

,Restaurante,Valoracion,Nota,Delivery_fee,ETA,Tag,Precio,Direccion,Barrio,Lat,Lon,Coordenates,type_point
0,Badsha India Restaurant,200,4.4,1.00,15–35,India,€,"Calle De Embajadores, Madrid",Centro,40.40069,-3.69925,"[-3.69925, 40.40069]","{'type': 'Point', 'coordinates': [-3.69925, 40..."
1,TUTTO PASTA!,26,2.8,2.50,40–60,Family Meals,€,"Calle De Núñez De Arce, 11, Madrid",Centro,40.41538,-3.70124,"[-3.70124, 40.41538]","{'type': 'Point', 'coordinates': [-3.70124, 40..."
2,Pizza Central,31,3.8,2.99,35–55,Pizza,€,"Plaza Del Emperador Carlos V, 10, Madrid",Centro,40.40861,-3.69288,"[-3.69288, 40.40861]","{'type': 'Point', 'coordinates': [-3.69288, 40..."
4,La Tagliatella,15,3.9,0.90,25–35,Pizza,€,"Calle De Toledo 27, Madrid",Centro,40.41377,-3.70737,"[-3.70737, 40.41377]","{'type': 'Point', 'coordinates': [-3.70737, 40..."
5,Fritz Foods,41,4.5,0.40,30–40,Pizza,€,"Calle De Luchana 17, Madrid",Centro,40.43000,-3.70099,"[-3.70099, 40.43]","{'type': 'Point', 'coordinates': [-3.70099, 40..."


In [10]:
type(df.type_point[0])

dict

In [11]:
type_point(df["Coordenates"][0])

{'type': 'Point', 'coordinates': [-3.69925, 40.40069]}

In [12]:
client = MongoClient("localhost:27017")

In [13]:
client = MongoClient("localhost:27017")
db = client.get_database("PROJECT")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROJECT')

In [14]:
collection = db["DarkKitchens"]

In [15]:
"""collection.create_index([("type_point_","2dsphere")])"""

'collection.create_index([("type_point_","2dsphere")])'

In [16]:
data = df.to_dict(orient = 'records')

In [17]:
data[0:1]

[{'Restaurante': 'Badsha India Restaurant',
  'Valoracion': 200,
  'Nota': 4.4,
  'Delivery_fee': 1.0,
  'ETA': '15–35',
  'Tag': 'India',
  'Precio': '€',
  'Direccion': 'Calle De Embajadores,   Madrid',
  'Barrio': 'Centro',
  'Lat': 40.40069,
  'Lon': -3.69925,
  'Coordenates': [-3.69925, 40.40069],
  'type_point': {'type': 'Point', 'coordinates': [-3.69925, 40.40069]}}]

In [18]:
#collection.insert_many(data)

In [19]:
df.isnull().sum()

Restaurante     0
Valoracion      0
Nota            0
Delivery_fee    0
ETA             0
Tag             0
Precio          0
Direccion       0
Barrio          0
Lat             0
Lon             0
Coordenates     0
type_point      0
dtype: int64

PARA PODER HACER GEO-NEAR NECESITO UN DATAFRAME CON:
 - Distrito
 - Dirección 
 - Latitud en función a la dirección de input para el web scrapping
 - Longitud en función a la dirección de input para el web scrapping
 - Typepoint de la dirección
 - Distancia en función del tamaño del distrito

In [20]:
distritos = ("Centro",
             "Arganzuela",
             "Retiro",
             "Salamanca",
             "Chamartín",
             "Tetuán",
             "Chamberí",
             "Moncloa-Aravaca",
             "Carabanchel",
             "Usera",
             "Puente de Vallecas",
             "Moratalaz",
             "Ciudad Lineal")

In [21]:
direcciones = ("Plaza Mayor, Madrid",
             "Calle de Cáceres, 37, Madrid",
             "Calle de Amado Nervo, 12, Madrid",
             "Calle de Don Ramón de la Cruz, 87, Madrid",
             "Calle de Honduras, 14, Madrid",
             "Calle de Bravo Murillo, 281, Madrid",
             "Calle de García de Paredes, 3, Madrid",
             "Calle de Juan del Rosal, 1, Madrid",
             "Calle General Ricardos, 134, Madrid",
             "Calle Antonio Velasco Zazo, 39, Madrid",
             "Calle Hermanos Trueba, 60, Madrid",
             "Calle del Camino de los Vinateros, 161, Madrid",
             "Calle Virgen del Castañar, 19, Madrid")

In [22]:
lists = (distritos,direcciones)
dist = pd.concat([pd.Series(x) for x in lists], axis=1)
dist

,0,1
0,Centro,"Plaza Mayor, Madrid"
1,Arganzuela,"Calle de Cáceres, 37, Madrid"
2,Retiro,"Calle de Amado Nervo, 12, Madrid"
3,Salamanca,"Calle de Don Ramón de la Cruz, 87, Madrid"
4,Chamartín,"Calle de Honduras, 14, Madrid"
5,Tetuán,"Calle de Bravo Murillo, 281, Madrid"
6,Chamberí,"Calle de García de Paredes, 3, Madrid"
7,Moncloa-Aravaca,"Calle de Juan del Rosal, 1, Madrid"
8,Carabanchel,"Calle General Ricardos, 134, Madrid"
9,Usera,"Calle Antonio Velasco Zazo, 39, Madrid"


In [23]:
"""columnas = ["distrito" ,"direccion","lat","lon","type_point","metros"]"""

'columnas = ["distrito" ,"direccion","lat","lon","type_point","metros"]'

In [24]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return {"type": "Point", "coordinates": None}

In [25]:
"""from geopy.geocoders import Nominatim
import time
geolocator = Nominatim(user_agent="Juancho")


def latitud_longitud(direccion):
    time.sleep(10)
    location = geolocator.geocode(direccion)
    print(location)
    return [location.latitude, location.longitude]"""

'from geopy.geocoders import Nominatim\nimport time\ngeolocator = Nominatim(user_agent="Juancho")\n\n\ndef latitud_longitud(direccion):\n    time.sleep(10)\n    location = geolocator.geocode(direccion)\n    print(location)\n    return [location.latitude, location.longitude]'

In [26]:
"""geolocator.geocode("Calle de Juan del Rosal 1, Madrid")"""

'geolocator.geocode("Calle de Juan del Rosal 1, Madrid")'

In [27]:
distancia = (600, 700, 600, 600, 1000, 600, 500, 1200, 1200, 800, 1200, 700, 1200)
lat_long = ([-3.70459,40.41646], [-3.6985513,40.3987617], [-3.6745702,40.4090564], [-3.67299,40.42851], [-3.6786952,40.4593232], [-3.6989388,40.4602247], [-3.69934,40.43682], [-3.69931,40.43182], [-3.7257475,40.3926858], [-3.7061415,40.3814357], [-3.66127,40.38738], [-3.64277,40.40619], [-3.64962,40.43927])
dist['metros'] = distancia
dist['lat_lon'] = lat_long
dist.columns = ["distrito","direccion","metros","lat_lon"]
dist

,distrito,direccion,metros,lat_lon
0,Centro,"Plaza Mayor, Madrid",600,"[-3.70459, 40.41646]"
1,Arganzuela,"Calle de Cáceres, 37, Madrid",700,"[-3.6985513, 40.3987617]"
2,Retiro,"Calle de Amado Nervo, 12, Madrid",600,"[-3.6745702, 40.4090564]"
3,Salamanca,"Calle de Don Ramón de la Cruz, 87, Madrid",600,"[-3.67299, 40.42851]"
4,Chamartín,"Calle de Honduras, 14, Madrid",1000,"[-3.6786952, 40.4593232]"
5,Tetuán,"Calle de Bravo Murillo, 281, Madrid",600,"[-3.6989388, 40.4602247]"
6,Chamberí,"Calle de García de Paredes, 3, Madrid",500,"[-3.69934, 40.43682]"
7,Moncloa-Aravaca,"Calle de Juan del Rosal, 1, Madrid",1200,"[-3.69931, 40.43182]"
8,Carabanchel,"Calle General Ricardos, 134, Madrid",1200,"[-3.7257475, 40.3926858]"
9,Usera,"Calle Antonio Velasco Zazo, 39, Madrid",800,"[-3.7061415, 40.3814357]"


In [28]:
unomas = []
for i in dist["lat_lon"]:
    unomas.append(type_point(i))
dist['type_point'] = unomas
dist

,distrito,direccion,metros,lat_lon,type_point
0,Centro,"Plaza Mayor, Madrid",600,"[-3.70459, 40.41646]","{'type': 'Point', 'coordinates': [-3.70459, 40..."
1,Arganzuela,"Calle de Cáceres, 37, Madrid",700,"[-3.6985513, 40.3987617]","{'type': 'Point', 'coordinates': [-3.6985513, ..."
2,Retiro,"Calle de Amado Nervo, 12, Madrid",600,"[-3.6745702, 40.4090564]","{'type': 'Point', 'coordinates': [-3.6745702, ..."
3,Salamanca,"Calle de Don Ramón de la Cruz, 87, Madrid",600,"[-3.67299, 40.42851]","{'type': 'Point', 'coordinates': [-3.67299, 40..."
4,Chamartín,"Calle de Honduras, 14, Madrid",1000,"[-3.6786952, 40.4593232]","{'type': 'Point', 'coordinates': [-3.6786952, ..."
5,Tetuán,"Calle de Bravo Murillo, 281, Madrid",600,"[-3.6989388, 40.4602247]","{'type': 'Point', 'coordinates': [-3.6989388, ..."
6,Chamberí,"Calle de García de Paredes, 3, Madrid",500,"[-3.69934, 40.43682]","{'type': 'Point', 'coordinates': [-3.69934, 40..."
7,Moncloa-Aravaca,"Calle de Juan del Rosal, 1, Madrid",1200,"[-3.69931, 40.43182]","{'type': 'Point', 'coordinates': [-3.69931, 40..."
8,Carabanchel,"Calle General Ricardos, 134, Madrid",1200,"[-3.7257475, 40.3926858]","{'type': 'Point', 'coordinates': [-3.7257475, ..."
9,Usera,"Calle Antonio Velasco Zazo, 39, Madrid",800,"[-3.7061415, 40.3814357]","{'type': 'Point', 'coordinates': [-3.7061415, ..."


In [29]:
dist.dtypes

distrito      object
direccion     object
metros         int64
lat_lon       object
type_point    object
dtype: object

VAMOS A EMPEZAR CON LAS GEO-NEAR POR DISTRITO

In [30]:
restaurants = db.get_collection("DarkKitchens")

In [31]:
df.Tag.unique()

array(['India', 'Family Meals', 'Pizza', 'Burritos', 'De Latinoamérica',
       'Empanada', 'Pollo', 'China', 'Hamburguesas', 'Italiana',
       'Vegetariano', 'Postres', 'Tienda', 'Americana', 'Halal',
       'Asiatica', 'Venezolana', 'Colombiana', 'Tapas y raciones',
       'Tapas', 'Alitas de pollo', 'Panadería', 'Comida rápida',
       'Cerveza', 'Especialidades', 'Bar Food', 'Mediterránea', 'Poke',
       'Carne', 'Platos a base de harina', 'Saludable', 'Pasta', 'Kebab',
       'Comida reconfortante', 'Gourmet', 'Everyday Essentials',
       'Croquetas', 'Ramen', 'Sushi', 'Alcohol', 'Opciones para alergias',
       'Mexicana', 'Árabe', 'Española', 'Queso', 'Con opciones veganas',
       'Cafe', 'Napolitana', 'Comfort Food', 'Sándwich', 'Bocadillo',
       'Parrilla', 'Boliviana', 'Asiática', 'Sudamericana', 'Peruana',
       'Fusión latina', 'Japonesa', 'Fritura', 'Mediterranea',
       'Fusión asiática', 'Fried Foods', 'Sándwiches'], dtype=object)

In [32]:
dist.head()

,distrito,direccion,metros,lat_lon,type_point
0,Centro,"Plaza Mayor, Madrid",600,"[-3.70459, 40.41646]","{'type': 'Point', 'coordinates': [-3.70459, 40..."
1,Arganzuela,"Calle de Cáceres, 37, Madrid",700,"[-3.6985513, 40.3987617]","{'type': 'Point', 'coordinates': [-3.6985513, ..."
2,Retiro,"Calle de Amado Nervo, 12, Madrid",600,"[-3.6745702, 40.4090564]","{'type': 'Point', 'coordinates': [-3.6745702, ..."
3,Salamanca,"Calle de Don Ramón de la Cruz, 87, Madrid",600,"[-3.67299, 40.42851]","{'type': 'Point', 'coordinates': [-3.67299, 40..."
4,Chamartín,"Calle de Honduras, 14, Madrid",1000,"[-3.6786952, 40.4593232]","{'type': 'Point', 'coordinates': [-3.6786952, ..."


In [33]:
toptags = ["Pizza","Hamburguesa","Pollo","Kebab","Empanada","Poke","Mexicano","Burritos"]

In [34]:
superlista =[]

for i,row in dist.iterrows():
    query_dist = {"Coordenates": {"$near": {"$geometry":{'type': 'Point', 'coordinates':dist.type_point[i]['coordinates']},
                                                  "$maxDistance": int(dist.metros[i])}}}
    restlist =(list(restaurants.find(query_dist)))
    
    dic_distr ={"Distrito":row["distrito"],"competidores": len(restlist),"Pizza":0,"Hamburguesa":0,"Pollo":0,"Kebab":0,"Empanada":0,"Poke":0,"Saludable":0,"Burritos":0,"Venezolana":0}
    
    for i in restlist:
        if i['Tag'] == 'Pizza':
            dic_distr["Pizza"] += 1
        elif i['Tag'] == 'Hamburguesas':
            dic_distr["Hamburguesa"] += 1
        elif i['Tag'] == 'Pollo':
            dic_distr["Pollo"] += 1
        elif i['Tag'] == 'Kebab':
            dic_distr["Kebab"] += 1
        elif i['Tag'] == 'Empanada':
            dic_distr["Empanada"] += 1
        elif i['Tag'] == 'Poke':
            dic_distr["Poke"] += 1
        elif i['Tag'] == 'Saludable':
            dic_distr["Saludable"] += 1
        elif i['Tag'] == 'Burritos':
            dic_distr["Burritos"] += 1
        elif i['Tag'] == 'Venezolana':
            dic_distr["Venezolana"] += 1
    
    superlista.append(dic_distr)    

In [48]:
darkkitchen = pd.DataFrame(superlista)

In [47]:
barrio = pd.read_csv("../Data/Barrios_cifras.csv")

In [44]:
pond_dark = pd.concat([barrio, darkkitchen.reindex(barrio.index)], axis=1)

In [46]:
darkitchen = pond_dark[["Superficie","Densidad","€ m2","competidores","Pizza","Hamburguesa","Pollo","Kebab","Empanada","Poke","Saludable","Burritos","Venezolana"]]
darkitchen.columns = ["Distrito","Densidad","Precio/m2","Competidores","Pizza","Hamburguesa","Pollo","Kebab","Empanada","Poke","Saludable","Burritos","Venezolana"]
darkitchen

,Distrito,Densidad,Precio/m2,Competidores,Pizza,Hamburguesa,Pollo,Kebab,Empanada,Poke,Saludable,Burritos,Venezolana
0,Centro,270,4865,43,4,5,6,0,4,0,0,1,1
1,Arganzuela,239,3888,38,13,2,2,1,1,1,1,0,1
2,Retiro,217,4502,8,2,0,1,1,2,1,1,0,0
3,Salamanca,271,5740,30,3,9,1,1,0,0,1,1,0
4,Chamartín,161,4999,5,0,0,2,0,1,0,0,0,0
5,Tetuán,297,3649,40,4,6,8,1,2,0,0,0,1
6,Chamberí,296,5283,10,0,1,1,0,0,0,0,0,0
7,Moncloa-Aravaca,26,3912,50,7,4,3,0,5,3,0,0,1
8,Carabanchel,184,2165,19,2,2,2,3,0,0,0,1,0
9,Usera,183,1942,10,0,0,2,0,0,0,0,1,2


In [82]:
tags = ["Pizza","Hamburguesa","Pollo","Kebab","Empanada","Poke","Saludable","Burritos","Venezolana"]
competidores = [darkitchen['Pizza'].sum(),
                darkitchen['Hamburguesa'].sum(),
                darkitchen['Pollo'].sum(),
                darkitchen['Kebab'].sum(),
                darkitchen['Empanada'].sum(),
                darkitchen['Poke'].sum(),
                darkitchen['Saludable'].sum(),
                darkitchen['Burritos'].sum(),
                darkitchen['Venezolana'].sum()]
nota = [df[df.Tag == 'Pizza']['Nota'].mean(),
        df[df.Tag == 'Hamburguesas']['Nota'].mean(),
        df[df.Tag == 'Pollo']['Nota'].mean(),
        df[df.Tag == 'Kebab']['Nota'].mean(),
        df[df.Tag == 'Empanada']['Nota'].mean(),
        df[df.Tag == 'Poke']['Nota'].mean(),
        df[df.Tag == 'Saludable']['Nota'].mean(),
        df[df.Tag == 'Burritos']['Nota'].mean(),
        df[df.Tag == 'Venezolana']['Nota'].mean()]
valoracion = [df[df.Tag == 'Pizza']['Valoracion'].mean(),
        df[df.Tag == 'Hamburguesas']['Valoracion'].mean(),
        df[df.Tag == 'Pollo']['Valoracion'].mean(),
        df[df.Tag == 'Kebab']['Valoracion'].mean(),
        df[df.Tag == 'Empanada']['Valoracion'].mean(),
        df[df.Tag == 'Poke']['Valoracion'].mean(),
        df[df.Tag == 'Saludable']['Valoracion'].mean(),
        df[df.Tag == 'Burritos']['Valoracion'].mean(),
        df[df.Tag == 'Venezolana']['Valoracion'].mean()]
knowledge = (6,6,6,3,4,5,4,3,4)


In [83]:
virtual = pd.DataFrame(list(zip(tags, competidores, nota, valoracion,knowledge)))
virtual.columns = ["Tag","Competidores","Avg Nota","Avg Valoraciones","Knowledge"]
virtual

,Tag,Competidores,Avg Nota,Avg Valoraciones,Knowledge
0,Pizza,40,4.059770,75.965517,6
1,Hamburguesa,32,4.013235,108.955882,6
2,Pollo,38,3.825000,60.562500,6
3,Kebab,11,4.013889,42.861111,3
4,Empanada,17,3.734286,31.285714,4
5,Poke,7,4.233333,127.583333,5
6,Saludable,3,3.771429,73.000000,4
7,Burritos,7,4.021429,38.857143,3
8,Venezolana,9,4.364706,109.764706,4
